In [4]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [5]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [6]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [7]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,_IbfoDr3Pb3d5z3gyGiC6yZEmncNb-vN6PIeboo1AfQ345U,TQa1E--XyuiMnpeEg6Huk5G4mFhbi__BiifAsViTb1a5EU0,IlLHb6mNyqf2N4VvRY3mh_qdisgyW6ToVq2pt110xgKS5W...,Wíthin,7,1656461956000,287,_IbfoDr3Pb3d5z3gyGiC6yZEmncNb-vN6PIeboo1AfQ345U,Wíthin,593,I,101,70,False,False,True,True
1,5PP1PByW13OTIHNs8f_MQbI2utzf-CKbFa3imrCUD-5JM-k,CE4kJ6tt7PwB4X24Y8CXS4pV73kqdSvJ0wWYyFi_ypioqA,SYeHG18d6HblklTVFAcDoGsAXWwTq1ZeflhRbaztiP900s...,prestivent,4026,1656492187000,128,5PP1PByW13OTIHNs8f_MQbI2utzf-CKbFa3imrCUD-5JM-k,prestivent,921,I,221,158,True,False,False,False
2,46WgJ4168aWmiVRTnSSXJ_wm6_7vtAwmnIUCes3iwPT9NfZr,Jitte0gYDlIrRoWWvhFLha2KedvBRgOWABNvvZVBzaMVnv...,pDoRFWQ6lYfIjHiwaOfyN3Gt4BtvI0zkIRMvB7418T2Xho...,rayditz,4656,1656481777000,66,46WgJ4168aWmiVRTnSSXJ_wm6_7vtAwmnIUCes3iwPT9NfZr,rayditz,765,I,152,118,False,False,True,False
3,iK7HHHIkQARvIiI98bNXpcmjRSyB1u4-KLuxPhbjHLPL5gc,_Y-IPIRIDt9C4KH7CpeJ4esQ-KssQ3Ongm2EjhEq8140hA,RfFChF-FG1121tg4frHGwr5UIeQWbjNwatqcVaVypMicfO...,QualityHooker,4641,1656420104000,224,iK7HHHIkQARvIiI98bNXpcmjRSyB1u4-KLuxPhbjHLPL5gc,QualityHooker,527,I,130,99,False,False,True,True
4,eMsAs5_WaPruim7bv3VhtHfBMltYhTQ9999-LcUVeU-cKYk,6duG4CESG_NemTQNUz_rdlNodWrgFO6Useh6XI6YOkbjTA,1mDiWGMTMZh3IFTiVjFXmhl53hq0hiHFitXwP_MBee5i1c...,Kyivix,547,1656468804000,146,eMsAs5_WaPruim7bv3VhtHfBMltYhTQ9999-LcUVeU-cKYk,Kyivix,693,I,112,72,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,K5dL326yBgAnSiwyYixn4B3Skk3Z6GnDP-k82NxogRrJ7R2P,E5Gb2tlteZ0q_2gILLF03OSC3TTYJvWgB0LF3oboxnY60z...,YBWyTuJ3jS4Z35UVvQ6yd0VzeaY94-oF8Dqcl7o1h09k8L...,zoe only champ,2,1656482274000,133,K5dL326yBgAnSiwyYixn4B3Skk3Z6GnDP-k82NxogRrJ7R2P,zoe only champ,623,I,121,91,False,False,True,False
92,JLgudRVr5LQ3QvehV_BLz-ep3HCwZ7w2SEcMDqFSmxTMA0U,-ynJs_256u41Ewk1CgnE2gNVT7RdMbCkqN6mP0xGJyjhO74,OigIlc5fb5-G8CarWNUGpMCjL4oy7GgsVsiUiQdB-Q_1Me...,RamKev,4636,1656445917000,271,JLgudRVr5LQ3QvehV_BLz-ep3HCwZ7w2SEcMDqFSmxTMA0U,RamKev,503,I,170,128,False,False,True,True
93,g60tNmXb8Uys9YkAs7zypog_qLMPINXTaePPn0ZT6NWMia19,PRN3LMWWmJ9fBwcYAorl93jHT68DSEYFp7W9i8wyNY4rED...,MOe2JAp9pd5PmwjIXhgZuTsF9qipi1EQ1WIj9qiXF9j5Ml...,ttv Clazzicola,29,1656512622529,1,g60tNmXb8Uys9YkAs7zypog_qLMPINXTaePPn0ZT6NWMia19,ttv Clazzicola,477,I,82,46,False,False,True,False
94,3TTmvP2TOYaLd-yGHjTSDrdB4fo9mgHg0Z3347_5mvEyLCE,M8z3qffZkrZoFzz6Nh6yOCHCI1RdH10QbAEO2rlO4nfFWiU,SIxmDzDxdLvM02pCXLiUI86YYgmqCozEeXBst7wGjAfen4...,casparwu,1114,1656503748000,85,3TTmvP2TOYaLd-yGHjTSDrdB4fo9mgHg0Z3347_5mvEyLCE,casparwu,906,I,175,131,False,False,True,False


In [8]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [9]:
len(uniq_matches_id)

3265

In [10]:
len(matches_asset)

4244

In [11]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [12]:
len(uniq_matches)

3265

In [13]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [14]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [15]:
len(latest_matches)

1393

In [16]:
# latest_matches[0]['info']

In [17]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [18]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4357324934',
 'placement': 3,
 'augment0': 'TFT7_Augment_PandorasBench',
 'augment1': 'TFT6_Augment_ItemGrabBag1',
 'augment2': 'TFT6_Augment_JeweledLotus',
 'Set7_Bruiser': 0,
 'Set7_Cavalier': 3,
 'Set7_Dragonmancer': 1,
 'Set7_Evoker': 0,
 'Set7_Guild': 1,
 'Set7_Jade': 0,
 'Set7_Legend': 1,
 'Set7_Mirage': 1,
 'Set7_Ragewing': 0,
 'Set7_Shimmerscale': 0,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'TFT7_Sejuani': 1,
 'TFT7_Nunu': 2,
 'TFT7_Nunu_item0': 'TFT_Item_Chalice',
 'TFT7_LeeSin': 2,
 'TFT7_LeeSin_item0': 'TFT_Item_RedBuff',
 'TFT7_LeeSin_item1': 'TFT_Item_LocketOfTheIronSolari',
 'TFT7_LeeSin_item2': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Volibear': 3,
 'TFT7_Volibear_item0': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Volibear_item1': 'TFT_Item_Quicksilver',
 'TFT7_Volibear_item2': 'TFT_Item_Bloodthirster',
 'TFT7_Anivia': 2,
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'TFT_Item_SparringGloves',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'TFT7_Item_CavalierEmblemItem',
 'TF

In [19]:
matches_league_df = pd.json_normalize(matches_array)

In [20]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cavalier,Set7_Dragonmancer,Set7_Evoker,Set7_Guild,...,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Skarner_item2,TFT7_Vladimir_item0,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Vladimir_item1,TFT7_Vladimir_item2
0,NA1_4357324934,3,TFT7_Augment_PandorasBench,TFT6_Augment_ItemGrabBag1,TFT6_Augment_JeweledLotus,0.0,3.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4357324934,5,TFT6_Augment_TrueTwos,TFT6_Augment_SecondWind1,TFT6_Augment_CelestialBlessing2,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4357324934,7,TFT6_Augment_RichGetRicher,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_PortableForge,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4357324934,4,TFT7_Augment_BruiserPersonalTraining,TFT6_Augment_SecondWind1,TFT6_Augment_PortableForge,1.0,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4357324934,2,TFT6_Augment_PortableForge,TFT7_Augment_JadeEmblem,TFT7_Augment_PandorasBench,0.0,NaN,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,NA1_4355410655,1,TFT7_Augment_JadeEmblem,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_ComponentGrabBag,NaN,NaN,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11140,NA1_4355410655,2,TFT6_Augment_CelestialBlessing1,TFT6_Augment_Weakspot,TFT7_Augment_AssassinEmblem,2.0,NaN,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11141,NA1_4355410655,6,TFT6_Augment_TinyTitans,TFT7_Augment_WarriorTrait,TFT6_Augment_Ascension,1.0,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11142,NA1_4355410655,3,TFT6_Augment_Electrocharge1,TFT7_Augment_BestFriends1,TFT6_Augment_ComponentGrabBag,NaN,0.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cavalier,Set7_Dragonmancer,Set7_Evoker,Set7_Guild,...,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Skarner_item2,TFT7_Vladimir_item0,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Vladimir_item1,TFT7_Vladimir_item2
0,NA1_4357324934,3,TFT7_Augment_PandorasBench,TFT6_Augment_ItemGrabBag1,TFT6_Augment_JeweledLotus,0.0,3.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4357324934,5,TFT6_Augment_TrueTwos,TFT6_Augment_SecondWind1,TFT6_Augment_CelestialBlessing2,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4357324934,7,TFT6_Augment_RichGetRicher,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_PortableForge,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4357324934,4,TFT7_Augment_BruiserPersonalTraining,TFT6_Augment_SecondWind1,TFT6_Augment_PortableForge,1.0,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4357324934,2,TFT6_Augment_PortableForge,TFT7_Augment_JadeEmblem,TFT7_Augment_PandorasBench,0.0,NaN,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,NA1_4355410655,1,TFT7_Augment_JadeEmblem,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_ComponentGrabBag,NaN,NaN,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11140,NA1_4355410655,2,TFT6_Augment_CelestialBlessing1,TFT6_Augment_Weakspot,TFT7_Augment_AssassinEmblem,2.0,NaN,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11141,NA1_4355410655,6,TFT6_Augment_TinyTitans,TFT7_Augment_WarriorTrait,TFT6_Augment_Ascension,1.0,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11142,NA1_4355410655,3,TFT6_Augment_Electrocharge1,TFT7_Augment_BestFriends1,TFT6_Augment_ComponentGrabBag,NaN,0.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)